In [0]:
%sql
select * from dev.dimdate


DateId,Year,MonthName


In [0]:
from datetime import datetime 
from pyspark.sql.functions import input_file_name, substring_index

parquet_file_path1 = "dbfs:/Silver/2_wheelers/202403/29/*.parquet"
parquet_file_path2 = "dbfs:/Silver/4_wheelers/202403/29/*.parquet"






In [0]:
#nnnn
from pyspark.sql.functions import col, input_file_name, lit, when, split, regexp_replace
import pyspark.sql.functions as F

stg_df_1 = spark.read.format('parquet').option('Header', 'True').option('inferSchema', 'True').load(parquet_file_path1) \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(F.col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(F.col('inputfilename') == '2-WheelSales', lit('F')).otherwise(lit('T'))).select(F.col('brand').alias('BrandName'), F.col('WheelerKey'))


stg_df_2 =  spark.read.format('parquet').option('Header', 'True').option('inferSchema', 'True').load(parquet_file_path2) \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(F.col('inputfilename') == '4-WheelSales', lit('T')).otherwise(lit('F'))).select(F.col('Brand').alias('BrandName'), F.col('WheelerKey'))

combined_stg = stg_df_1.unionByName(stg_df_2)
print("combined_count :", combined_stg.count())
pattern = " "
# combined_stg = combined_stg.withColumn("BrandName", regexp_replace(col("BrandName"), pattern, "")).dropDuplicates(['BrandName', 'WheelerKey'])
combined_stg = combined_stg.dropDuplicates(['BrandName', 'WheelerKey'])

print("combined_distinct :" ,combined_stg.count())
combined_stg.show()

dim_df = spark.sql("""SELECT * FROM  dev.dimbrand""")
print('dim_df show...')
dim_df.show()
print("check for new records...")

new_records = combined_stg.join(dim_df, 'BrandName', 'left_anti')
new_records.show()

updated_records = combined_stg.join(dim_df, combined_stg['BrandName'] == dim_df['BrandName'], 'inner').select(dim_df['*'])

print("matching from src to target ")
updated_records.show()

combined_records = new_records.unionByName(updated_records.drop('pkBrandId'))

print('combined....')
combined_records = combined_records.dropDuplicates(['BrandName', 'WheelerKey'])
print('combined_records df count:', combined_records.count())
combined_records.show()

combined_records.write.mode('overwrite').saveAsTable("dev.dimbrand")
print("success") 

combined_count : 116786
combined_distinct : 34
+-------------+----------+
|    BrandName|WheelerKey|
+-------------+----------+
|          TVS|         T|
|       Yamaha|         T|
|      Piaggio|         T|
|        Honda|         T|
|      Okinawa|         T|
|Royal Enfield|         T|
|Hero MotoCorp|         T|
|        Bajaj|         T|
|       Jaguar|         F|
|Mercedes Benz|         F|
|          BMW|         F|
|         Audi|         F|
|        Lexus|         F|
|      Ferrari|         F|
|  Rolls Royce|         F|
|     Maserati|         F|
|  Lamborghini|         F|
|      Porsche|         F|
|        Volvo|         F|
|        Honda|         F|
+-------------+----------+
only showing top 20 rows

dim_df show...
+---------+---------+----------+
|pkBrandId|BrandName|WheelerKey|
+---------+---------+----------+
+---------+---------+----------+

check for new records...
+-------------+----------+
|    BrandName|WheelerKey|
+-------------+----------+
|          TVS|         T

In [0]:
%sql
select * from dev.dimbrand

pkBrandId,BrandName,WheelerKey
1,TVS,T
2,Yamaha,T
3,Piaggio,T
4,Honda,T
5,Okinawa,T
6,Royal Enfield,T
7,Hero MotoCorp,T
8,Bajaj,T
9,Jaguar,F
10,Mercedes Benz,F


In [0]:
%sql
truncate table dev.dimbrand

In [0]:
# from pyspark.sql.functions import *
# from pyspark.sql.types import *
# from datetime import datetime 

# def Brand_scd1(path1, path2):
#     # Read data from Parquet files and add necessary columns
#     stg_df_2 = spark.read.format('parquet') \
#                     .option('Header', 'True') \
#                     .load(path1) \
#                     .withColumn('inputfilename', input_file_name()) \
#                     .withColumn('inputfilename',  concat_ws("/", expr("slice(split(inputfilename, '/'), 1, 3)"))) \
#                     .withColumn('wheelerkey', when(col('inputfilename') == 'dbfs:/Silver/2_wheelers', lit('T')).otherwise(lit('F'))) \
#                     .select(col('Brand').alias('BrandName'), 'wheelerkey').distinct()

#     stg_df_4 = spark.read.format('parquet') \
#                     .option('Header', 'True') \
#                     .load(path2) \
#                     .withColumn('inputfilename', input_file_name()) \
#                     .withColumn('inputfilename',  concat_ws("/", expr("slice(split(inputfilename, '/'), 1, 3)"))) \
#                     .withColumn('wheelerkey', when(col('inputfilename') == 'dbfs:/Silver/4_wheelers', lit('F')).otherwise(lit('T'))) \
#                     .select(col('Brand').alias('BrandName'), 'wheelerkey').distinct()

#     # # Load Delta table
#     # delta_table = DeltaTable.forPath(spark, "dbfs:/delta_tables/Dim1")

#     combined_stg = stg_df_2.unionByName(stg_df_4)

#     # combined_stg.show()

#     print("read the table")

#     dim_df = spark.sql("select * from dev.dimbrand")

#     # dim_df.show()
#     print("check for new and updated records...")
#     new_records = combined_stg.join(dim_df, 'BrandName', 'left_anti')
#     new_records.show()

#     print("check for matched records")

#     matched_records = combined_stg.join(dim_df,  combined_stg['BrandName']==dim_df['BrandName'], 'inner').select(combined_stg['*'])

#     matched_records.show()

#     print("final_df")

#     final_df = matched_records.drop('pkBrandId').unionByName(new_records)

#     # final_df.show()

#     final_df.write.mode('overwrite').saveAsTable('dev.dimbrand')

#     print('success')


    

In [0]:
Brand_scd1(parquet_file_path1, parquet_file_path1)


read the table
check for new and updated records...
+-------------+----------+
|    BrandName|wheelerkey|
+-------------+----------+
|          TVS|         T|
|       Yamaha|         T|
|      Piaggio|         T|
|        Honda|         T|
|      Okinawa|         T|
|Royal Enfield|         T|
|Hero MotoCorp|         T|
|        Bajaj|         T|
|          TVS|         T|
|       Yamaha|         T|
|      Piaggio|         T|
|        Honda|         T|
|      Okinawa|         T|
|Royal Enfield|         T|
|Hero MotoCorp|         T|
|        Bajaj|         T|
+-------------+----------+

check for matched records
+---------+----------+
|BrandName|wheelerkey|
+---------+----------+
+---------+----------+

final_df
success


In [0]:
%sql
truncate table dev.dimbrand

In [0]:
# p = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('dbfs:/FileStore/4_wheelers/BRAND_TEST2.csv')
# fp = "dbfs:/FileStore/4_wheelers/BRAND_TEST2.parquet"
# p.write.mode("overwrite").parquet(fp)


In [0]:
#nnnn_test
stg_df_1 = spark.read.format('csv').option('Header', 'True').option('inferSchema', 'True').load('dbfs:/FileStore/4_wheelers/2_WheelSales_scd1_test.csv') \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(col('inputfilename') == '2-WheelSales', lit('F')).otherwise(lit('T'))).select(F.col('OEM').alias('BrandName'), F.col('WheelerKey'))


stg_df_2 =  spark.read.format('csv').option('Header', 'True').option('inferSchema', 'True').load('dbfs:/FileStore/4_wheelers/4_WheelSales_scd1_test.csv') \
    .withColumn('inputfilename', input_file_name()).withColumn('inputfilename', split(col('inputfilename'), '/').getItem(2)).withColumn('WheelerKey', when(col('inputfilename') == '4-WheelSales', lit('T')).otherwise(lit('F'))).select(F.col('Brand').alias('BrandName'), F.col('WheelerKey'))

combined_stg = stg_df_1.unionByName(stg_df_2)
print("combined_count :", combined_stg.count())
pattern = " "
combined_stg = combined_stg.withColumn("BrandName", regexp_replace(col("BrandName"), pattern, "")).dropDuplicates(['BrandName', 'WheelerKey'])
print("combined_distinct :" ,combined_stg.count())
combined_stg.show()

dim_df = spark.sql("""SELECT * FROM  dev.dimbrand""")
print('dim_df show...')
dim_df.show()
print("check for new records...")

new_records = combined_stg.join(dim_df, 'BrandName', 'left_anti')
new_records.show()

updated_records = combined_stg.join(dim_df, combined_stg['BrandName'] == dim_df['BrandName'], 'inner').select(dim_df['*'])

print("matching from src to target ")
updated_records.show()

combined_records = new_records.unionByName(updated_records.drop('pkBrandId'))

# combined_records = combined_records.withColumn("BrandName", regexp_replace("BrandName", " +", " "))

print('combined....')
combined_records = combined_records.dropDuplicates(['BrandName', 'WheelerKey'])
print('combined_records df count:', combined_records.count())
combined_records.show()

combined_records.write.mode('overwrite').saveAsTable("dev.dimbrand")

print("success")

combined_count : 116786
combined_distinct : 34
+------------+----------+
|   BrandName|WheelerKey|
+------------+----------+
|         TVS|         T|
|RoyalEnfield|         T|
|      Yamaha|         T|
|     Piaggio|         T|
|       Honda|         T|
|     Okinawa|         T|
|HeroMotoCorp|         T|
|    BajajOne|         T|
|      Jaguar|         F|
|         BMW|         F|
|        Audi|         F|
|       Lexus|         F|
|     Ferrari|         F|
|MercedesBenz|         F|
|  RollsRoyce|         F|
|  PorscheNew|         F|
|       Volvo|         F|
|       Honda|         F|
|     Hyundai|         F|
|        Jeep|         F|
+------------+----------+
only showing top 20 rows

dim_df show...
+---------+-------------+----------+
|pkBrandId|    BrandName|WheelerKey|
+---------+-------------+----------+
|       69|          TVS|         T|
|       70|       Yamaha|         T|
|       71|      Piaggio|         T|
|       72|        Honda|         T|
|       73|      Okinawa|    

In [0]:
%sql
select * from dev.dimbrand

pkBrandId,BrandName,WheelerKey
103,RoyalEnfield,T
104,MINICooper,F
105,MercedesBenz,F
106,HeroMotoCorp,T
107,RollsRoyce,F
108,BajajOne,T
109,PorscheNew,F
110,Jaguar,F
111,Honda,F
112,Renault,F
